DATA PREPROCESSING FILE

In [2]:
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Masking, Reshape
from sklearn.model_selection import train_test_split
from scipy.sparse import hstack

In [3]:
#Define constant 

RNA_BASES = [["A"], ["U"], ["C"], ["G"]]
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False).fit(RNA_BASES)

In [ ]:
def get_arguments():
    """
    Parse command-line arguments.
    Returns:
        argparse.Namespace: An object containing the parsed command-line arguments.
    """
    parser = argparse.ArgumentParser(description="Data Preprocessing Script")
    parser.add_argument('--input_file', required=True, help="Path to the input CSV file")
    parser.add_argument('--output_file', required=True, help="Path to save the processed data")

    args = parser.parse_args()
    return args

In [4]:
def load_data(file_path) :
    
    """
    Load data from a CSV file.

    Parameters:
    - file_path (str): The path to the CSV file containing the data.

    Returns:
    - data(DataFrame): A Pandas DataFrame containing the loaded data.
    """
    data = pd.read.csv(file_path)
    return data

In [5]:
def filter_SN(data):
    
    """ 
    Filter the rows where "SN_filter" is equal to 1 
       
    Parameters :
    -data(DataFrame) : A pandas Dataframe containing the input data
    
    Returns :
    - cleaned_train_data(DataFrame) : A dataframe containing only the sequences that passed the SN_filter 
    """
    cleaned_train_data = data[data['SN_filter'] == 1]
    return cleaned_train_data

In [6]:
def filter_identical_sequences(data):
    
    """
    For identical sequences keep rows with maximum signal to noise
    
    Parameters:
    -data (DataFrame) : pandas input dataframe
    
    Returns :
    -filtered_df(Dataframe) : a filtered dataframe with the sequences with a maximum signal to noise
    """
    
    filtered_df = cleaned_train_data.groupby('sequence').apply(lambda x: x.loc[x['signal_to_noise'].idxmax()])
    return filtered_df

In [7]:
def columns_defining(cleaned_train_data) :
    
    """
    Define the X and Y columns
    
    Parameters:
    -cleaned_trained_data (Dataframe) : pandas dataframe input
    
    Returns :
    -x_columns(list) : column names for X
    -y_columns(list) : column names for Y
    -conditional_columns(list) : continonal columns for the whole dataset
    """
    
    x_columns = ["sequence_id", "sequence"]
    conditional_columns = ["experiment_type", "signal_to_noise"]
    y_columns = [colname for colname in cleared_train_data.columns if re.match("^reactivity_[0-9]{4}$", colname)]
    return x_columns, y_columns, conditional_columns

In [8]:
def column_filtering(cleaned_train_data, x_columns, y_columns, conditional_columns) :
    
    """
    Select the necessary columns in the dataframe
    
    Parameters :
    -cleaned_train_data(DataFrame) : dataframe input
    -x_columns(list): list of columns for X
    -y_columns(list): list of columns for Y
    -conditional_columns(list): list of conditional columns
    
    Returns : 
    -cleaned_train_data(DataFrame) : dataframe with X and Y columns and the conditional ones
    """
    
    cleaned_train_data = cleaned_train_data[x_columns + conditional_columns + y_columns]
    return cleaned_train_data

In [9]:
def dataframe_separation(cleaned_train_data):
    
    """
    Create two dataframe based on the experiment type (DMS and 2A3)
    
    Parameters :
    -cleaned_train_data(DataFrame) : input dataframe to be separated in two
    
    Returns :
    -df_2A3_MaP(DataFrame) : dataframe with the 2A3 results only
    -df_DMS_MaP(DataFrame) : dataframe with the DMS results only
    
    """
    df_2A3_MaP = cleaned_train_data[cleaned_train_data['experiment_type'] == '2A3_MaP']
    df_DMS_MaP = cleaned_train_data[cleaned_train_data['experiment_type'] == 'DMS_MaP']
    
    # Delete cleaned_train_data to free space memory
    del cleaned_train_data
    
    return df_2A3_MaP, df_DMS_MaP 

In [10]:
def dataframe_concatenation(df_2A3_MaP, df_DMS_MaP) : 
    
    """
    Concatenation of the experiment dataframes
    
    Parameters :
    - df_2A3_MaP(DataFrame) : dataframe with the 2A3 experiment results
    - df_DMS_MaP(DataFrame) : dataframe with the DMS experiment results
    
    Returns :
    
    -cleared_train_data(DataFrame) : concatenated dataframe
    """
    
    mask_2A3 = df_2A3_MaP["sequence"].isin(df_DMS_MaP["sequence"])
    mask_DMS = df_DMS_MaP["sequence"].isin(df_2A3_MaP["sequence"])
    
    cleared_train_data = pd.concat([df_2A3_MaP[mask_2A3], df_DMS_MaP[mask_DMS]], ignore_index=True)
    
    return cleared_train_data

In [11]:
def reactivaty_normalization(df_2A3_MaP, df_DMS_MaP) :
    """
    Robust Z-score Normalization of reactivities 
    
    Parameters :
    
    - df_2A3_MaP(DataFrame) : dataframe with the 2A3 experiment results
    - df_DMS_MaP(DataFrame) : dataframe with the DMS experiment results
    
    Returns :
    
    - df_2A3_MaP(DataFrame) : Normalized dataframe with the 2A3 experiment results
    - df_DMS_MaP(DataFrame) : Normalized dataframe with the DMS experiment results
    """
    
    # Calculate median and Median Absolute Deviation (MAD) for robust normalization
    dms_median = np.nanmedian(df_2A3_MaP)
    a3_median = np.nanmedian(df_DMS_MaP)
    dms_mad = np.nanmedian(np.abs(np.array(df_DMS_MaP) - dms_median))
    a3_mad = np.nanmedian(np.abs(np.array(df_2A3_MaP) - a3_median))

    # Print the calculated values
    print("dms_median:", dms_median)
    print("a3_median:", a3_median)
    print("dms_mad:", dms_mad)
    print("a3_mad:", a3_mad)

    # Apply robust z-score normalization to all DataFrames
    for i in range(len(df_2A3_MaP)):
        df_DMS_MaP[i]['DMS_MaP_Reactivity'] = (df_DMS_MaP[i]['DMS_MaP_Reactivity'] - dms_median) / (1.482602218505602 * dms_mad)
     
    for i in range(len(df_DMS_MaP)):    
        df_2A3_MaP[i]['2A3_DMS_Reactivity'] = (df_2A3_MaP[i]['2A3_MaP_Reactivity'] - a3_median) / (1.482602218505602 * a3_mad)

    
    return df_DMS_MaP, df_2A3_MaP
    
    
    

In [12]:
def index_reset(cleared_train_data) : #anis data
    """
    Reset the index of a DataFrame and add the old index as a new column.

    Parameters:
    cleared_train_data(pd.DataFrame): The DataFrame to reset the index of.

    Returns:
    None
    """
    
    cleared_train_data.reset_index(drop=True, inplace=True)

In [13]:
def rm_signal_to_noise(cleared_train_data) : #roude data
    """
    Drop signal to noise
    
    Parameters :
    - cleared_train_data(pd.Dataframe) : Dataframe to be filtered
    
    Returns :
    
    - cleared_train_data(pd.Dataframe) : Dataframe without signal_to_noise_column
    
    """
    
    cleared_train_data = cleared_train_data.drop(columns=['signal_to_noise'])
    
    return cleared_train_data
     

In [14]:
def extraction(data) : #may need to be removed
    
    """
    Extract RNA sequences and experiment type
    
    Parameters :
    - data(Dataframe) : preloaded dataframe with the right format
    
    Returns:
    - rna_sequence(Series) : a pandas series with all the sequences
    - experiment_type(Series) : a panda series with the experiment types
    """
    
    rna_sequences = data['sequence']
    experiment_type = data['experiment_type']
    
    return rna_sequence, experiment_type

In [15]:
def features_encoding(rna_sequences, experiment_type) : #may need to be removed
    
    """
    Fit and transform RNA sequences
    
    Parameters :
    - rna_sequence(Series) : a pandas series with all the RNA sequences
    - experiment_type(Series) : a panda series with the experiment types
    
    Returns :
    - features(matrix) : concatenated encoded matrix
    
    """
    
    encoder = OneHotEncoder(sparse_output = True, dtype=np.int64)
    rna_sequences_encoded = encoder.fit_transform(rna_sequences.values.reshape(-1,1))
    experiment_type_encoded = pd.get_dummies(experiment_type)
    features = hstack((rna_sequences_encoded, experiment_type_encoded))
    return features


In [16]:
def get_target(cleared_train_data) : #may need to be removed
    
    """
    Extract reactivity columns as targets to use as Y
    
    Parameters :
    cleared_train_data(DataFrame) : pandas dataframe with the whole data
    
    Returns : 
    targets(DataFrame) : dataframe with reactivity columns
    """
    
    reactivity_columns = cleared_train_data.columns[~cleared_train_data.columns.isin(['sequence','experiment_type'])]
    targets = cleared_train_data[reactivity_columns]
    
    return targets

In [17]:
def reshape_inp(X_train, X_val) : #may need to be removed
    """
    Reshape the input format

    Parameters:
    - X_train: Training feature matrix (e.g., dense matrix)
    - X_val: Validation feature matrix (e.g., dense matrix)

    Returns:
    - X_train_reshaped: Reshaped training data with time step dimension
    - X_val_reshaped: Reshaped validation data with time step dimension
    """
    
    #Convert dense matrix to dense array 
    
    X_train_dense = X_train.toarray()
    X_val_dense = X_val.toarray()
    
    # Reshape input data to include the time step dimension
    timesteps = 1  # Number of time steps (since since we have masked sequences)
    input_dim = X_train_dense.shape[1]
    X_train_reshaped = X_train_dense.reshape(X_train_dense.shape[0], timesteps, input_dim)
    X_val_reshaped = X_val_dense.reshape(X_val_dense.shape[0], timesteps, input_dim)
    
    return X_val_reshaped, X_val_reshaped

In [18]:
def onehot_from_sequence(sequence, encoder, to_add="0", maxlen=457):
    """
    Encoder and padder for sequences :
    Parameters :
    sequence (str) : sequence of nucleotides to be encoded
    encoder () : encoder from keras
    
    Returns :
    one_hot_sequence() : encoded and padded sequence
    """
    proccessed_sequence = sequence.upper()
    proccessed_sequence += to_add * (maxlen - len(sequence)) #padding sequence
    proccessed_sequence = [[nbase] for nbase in proccessed_sequence]
    onehot_sequence = encoder.transform(proccessed_sequence)
    return onehot_sequence

In [19]:
def padded_matrix(matrix_2d, maxlen=457):
    """
    Padder for Y values
    Parameters :
    - matrix_2D(matrix) : Matrix to be padded
    Returns :
    - matrix_2S_padded(matrix) : Padded matrix
    """
    
    if not isinstance(matrix_2d, np.ndarray):
        matrix_2d = np.array(matrix_2d)

    n_toadd = maxlen - matrix_2d.shape[0] #number of n to add
    padding = ((0, n_toadd), (0, 0))  # padding on axis
    matrix_2d_padded = np.pad(matrix_2d, pad_width=padding, mode="constant")
    return matrix_2d_padded

In [20]:
def get_XY(seq_reactivity, encoder, maxlen=457) :
    
    """
    Encoding X and padding Y
    Parameters :
    dict_data(Dictionnary) : dictionnary issued from cleared data
    encoder : encoder 
    Returns :
    x_list(np array) : encoded sequences
    y_list(np array) : padded reactivities
    """
    
    x_list = []
    y_list = []
    for i, (sequence, reactivities) in enumerate(dict_data.items()):
        y = np.hstack([reactivities["2A3_MaP"], reactivities["DMS_MaP"]])
        x_list.append(onehot_from_sequence(sequence, encoder, maxlen=maxlen))
        y_list.append(padded_matrix(y, maxlen=maxlen))
    return np.array(x_list), np.array(y_list)
    

In [21]:
def get_seq_reactivity(cleared_train_data, keys_name=["2A3_MaP", "DMS_MaP"]) :
    """
    Parameters (DataFrame) : input dataframe to extract reactivities from
    - data (DataFrame) : input dataframe to extract reactivities from
    
    Returns :
    seq_reactivity (Dictionnary) : reactivities for each sequence
    """
    
    n_duplicate = sum(data.duplicated(subset=["sequence", "experiment_type"]))
    if n_duplicate > 0:
        return None
    
    seq_reactivity = dict()
    for seq, group in data.groupby("sequence"):
        seq_reactivity[seq] = dict()
        for key in keys_name :
            mask = group["experiment_type"] == key
            seq_reactivity[seq][key] = group[mask].drop(
                labels=["sequence", "experiment_type"], axis=1
            ).values.reshape(-1)
            seq_reactivity[seq][key] = np.expand_dims(seq_reactivity[seq][key], axis=1)

    return seq_reactivity

In [22]:
def csv_export(cleared_train_data, y_columns) :
    
    """
    Export the dataframe into a csv file
    
    Parameters : 
    - cleared_train_data(DataFrame) : dataframe with the whole data
    - y_columns(list) : columns for Y data
    """
    
    #Convert the y columns     
    cleared_train_data[y_columns] = cleared_train_data[y_columns].astype(np.float32)
    
    #export the csv
    csv_path = input("Enter your path :") 
    cleared_train_data.to_csv(csv_path, index=False)
    

In [23]:
def reactivity_masking(y) : 
    
    """
    Handle Na reactivity values by creating a mask
    
    Parameters : 
    y(DataFrame) : dataframe with reactivity columns
    
    Returns :
    reactivity_mask(Boolean mask) : mask for na reactivity values
    """
    
    x_mask = x.sum(axis=2) == 0
    reactivity_mask = ~np.isnan(y.values)
    return reactivity_mask

In [24]:
def train_val_sets(features, targets, reactivity_mask): 
    
    """
    Creates the Validation and the training sets
    
    Parameters :
    - features(matrix) : concatenated encoded matrix
    - targets(DataFrame) : dataframe with reactivity columns
    - reactivity_mask(Boolean mask) : mask for na reactivity values
    
    Returns :
    
    - X_train
    - X_val
    - y_train
    - y_val
    - mask_train
    - mask_val
    """
    
    X_train, X_val, y_train, y_val, mask_train, mask_val = train_test_split(
        features, targets, reactivity_mask, test_size=0.2, random_state=42)
    
    return X_train, X_val, y_train, y_val, mask_train, mask_val

ANIS DATASET

In [25]:
def remove_columns(data) : 
    """
    Remove unnecessary columns
    Parameters :
    -data(dataframe) : dataset of the data
    Returns :
    -cleaned_train_data(dataframe) : cleaned dataset without useless columns
    """
    
    # List of columns to remove
    columns_to_remove = ["sequence_id", "dataset_name", "reads", "SN_filter"]

    # Find columns containing "reactivity_error" in their names
    reactivity_error_columns = [col for col in data.columns if "reactivity_error" in col]

    # Combine the columns to remove
    columns_to_remove.extend(reactivity_error_columns)

    # Drop the specified columns from the DataFrame
    cleaned_train_data = data.drop(columns=columns_to_remove)

In [26]:
def pandas_dataset_converting(data) :
    """
    Parameters :
    - data(dataframe) : original_dataset
    Returns : 
    - computed_data (dataframe) : pandas dataframe
    """
    
    computed_data = data.compute()
    return computed_data

In [27]:
def reduce_mem_usage(df, verbose=True):
    """
    Reduce memory usage 
    Parameters :
    -df(Dataframe) : dataframe with important memory
    Returns :
    -df(Dataframe) : dataframe with reduced dimensions
    """
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [28]:
# Define a function to process the entire DataFrame
def process_dataframe(df):
    """
    Process the whole dataframe 
    Parameters :
    -df(Dataframe) : Dataframe with important data
    Return :
    -array_list(np.array) : input for the model
    """
    
    result_list = []
    unique_sequences = df['sequence'].unique() #filter unique sequences

    # Create empty lists to store values for robust normalization
    dms_values = []
    a3_values = []

    for sequence in unique_sequences:
        sequence_data = df[df['sequence'] == sequence]
        dms_map_row = sequence_data[sequence_data['experiment_type'] == 'DMS_MaP']
        a3_map_row = sequence_data[sequence_data['experiment_type'] == '2A3_MaP']

        tensor_rows = []
        for i in range(len(sequence)):
            dms_value = dms_map_row[f'reactivity_{i+1:04d}'].iloc[0] if not dms_map_row.empty else np.nan
            a3_value = a3_map_row[f'reactivity_{i+1:04d}'].iloc[0] if not a3_map_row.empty else np.nan
            nucleotide = sequence[i]
            tensor_row = [nucleotide, dms_value, a3_value]
            tensor_rows.append(tensor_row)

            # Append values for robust normalization, only if they are not NaN
            if not np.isnan(dms_value):
                dms_values.append(dms_value)
            if not np.isnan(a3_value):
                a3_values.append(a3_value)

        # Create a DataFrame from the tensor_rows
        tensor_df = pd.DataFrame(tensor_rows, columns=['Nucleotide', 'DMS_MaP_Reactivity', '2A3_MaP_Reactivity'])

        # Perform one-hot encoding for the 'Nucleotide' column
        nucleotide_encoded = pd.get_dummies(tensor_df['Nucleotide'], prefix='Nucleotide')

        # Drop the original 'Nucleotide' column
        tensor_df.drop(columns=['Nucleotide'], inplace=True)

        # Concatenate the one-hot encoded columns with the original DataFrame
        tensor_df = pd.concat([nucleotide_encoded, tensor_df], axis=1)

        result_list.append(tensor_df)

    # Calculate median and Median Absolute Deviation (MAD) for robust normalization
    dms_median = np.nanmedian(dms_values)
    a3_median = np.nanmedian(a3_values)
    dms_mad = np.nanmedian(np.abs(np.array(dms_values) - dms_median))
    a3_mad = np.nanmedian(np.abs(np.array(a3_values) - a3_median))

    # Print the calculated values
    print("dms_median:", dms_median)
    print("a3_median:", a3_median)
    print("dms_mad:", dms_mad)
    print("a3_mad:", a3_mad)

    # Apply robust z-score normalization to all DataFrames
    for i in range(len(result_list)):
        result_list[i]['DMS_MaP_Reactivity'] = (result_list[i]['DMS_MaP_Reactivity'] - dms_median) / (1.482602218505602 * dms_mad)
        result_list[i]['2A3_MaP_Reactivity'] = (result_list[i]['2A3_MaP_Reactivity'] - a3_median) / (1.482602218505602 * a3_mad)

    # Convert each data frame into a NumPy array
    array_list = [df.to_records(index=False) for df in result_list]

    return array_list